In [1]:
import time, numpy as np

**test examples first!**

**beware edge cases**

9a

In [41]:
DVECS={
    'R':(1,0),
    'L':(-1,0),
    'U':(0,1),
    'D':(0,-1)
}
MOORE_DVECS=list(DVECS.values())+[
    (1,1),
    (1,-1),
    (-1,1),
    (-1,-1)
]
vsum=lambda a,b:tuple([ai+bi for ai,bi in zip(a,b)])
diffs=lambda a,b:set(abs(ai-bi) for ai,bi in zip(a,b))
mdist=lambda a,b:sum(abs(ai-bi) for ai,bi in zip(a,b))
chdist=lambda a,b:max(abs(ai-bi) for ai,bi in zip(a,b))
with open('in.txt') as f:
    instrs=[]
    for l in f.read().split('\n'):
        d,v=l.split(' ')
        instrs.append((d,int(v)))
all_tails=set()
head,tail=(0,0),(0,0)
all_tails.add(tail)
for d,rep in instrs:
    #print('***')
    for _ in range(rep):
        vec=DVECS[d]
        head=vsum(head,vec)
        if diffs(head,tail)=={2,0}:
            tail=vsum(tail,vec)
        elif chdist(head,tail)>1:
            best_moore=sorted(MOORE_DVECS,
                              key=lambda v:mdist(vsum(tail,v),head)
                             )
            #print([(v,mdist(v,head)) for v in best_moore])
            best_moore=best_moore[0]
            tail=vsum(tail,best_moore)
        all_tails.add(tail)
        #print(head,tail,vec)
len(all_tails)

5907

9b

In [40]:
DVECS={
    'R':(1,0),
    'L':(-1,0),
    'U':(0,1),
    'D':(0,-1)
}
vsum=lambda a,b:tuple([ai+bi for ai,bi in zip(a,b)])
diffs=lambda a,b:set(abs(ai-bi) for ai,bi in zip(a,b))
mdist=lambda a,b:sum(abs(ai-bi) for ai,bi in zip(a,b))
chdist=lambda a,b:max(abs(ai-bi) for ai,bi in zip(a,b))
def cap(v):
    return max(-1,min(1,v))
def ntail(head,tail):
    if chdist(head,tail)>1:
        return vsum(tail,
                    (cap(head[0]-tail[0]),
                     cap(head[1]-tail[1]))
                   )
    else:
        return tail
with open('in.txt') as f:
    instrs=[]
    for l in f.read().split('\n'):
        d,v=l.split(' ')
        instrs.append((d,int(v)))
all_tails=set()
rope=[(0,0) for _ in range(10)]
all_tails.add(rope[-1])
for d,rep in instrs:
    #print('***')
    for _ in range(rep):
        vec=DVECS[d]
        rope[0]=vsum(rope[0],vec)
        for i in range(1,len(rope)):
            rope[i]=ntail(rope[i-1],rope[i])
        all_tails.add(rope[-1])
    #print(d,rep,rope)
len(all_tails)

2303

8a

In [6]:
with open('in.txt') as f:
    A=list(map(list,f.read().split('\n')))
    A=[list(map(int,row)) for row in A]
#print(A)
R,C=len(A),len(A[0])
DR,DC=[1,0,-1,0],[0,1,0,-1]
out=0
for i in range(R):
    for j in range(C):
        vis=False
        for d in range(4):
            v=A[i][j]
            vals=[]
            ni,nj=i,j
            while 0<=ni and ni<R and 0<=nj and nj<C:
                if (ni,nj)!=(i,j):
                    vals.append(A[ni][nj])
                ni+=DR[d]
                nj+=DC[d]
            if len(vals)==0 or max(vals)<v:
                vis=True
                break
        if vis:
            out+=1
print(out)

1684


8b

In [17]:
with open('in.txt') as f:
    A=list(map(list,f.read().split('\n')))
    A=[list(map(int,row)) for row in A]
#print(A)
R,C=len(A),len(A[0])
DR,DC=[1,0,-1,0],[0,1,0,-1]
out=0
def prod(L):
    out=1
    for v in L:
        out*=v
    return out
for i in range(R):
    for j in range(C):
        dists=[]
        for d in range(4):
            v=A[i][j]
            ni,nj=i,j
            l=0
            while 0<=ni and ni<R and 0<=nj and nj<C:
                if (ni,nj)!=(i,j):
                    l+=1
                    nv=A[ni][nj]
                    if nv>=v:
                        break
                ni+=DR[d]
                nj+=DC[d]
            dists.append(l)
        #print(i,j,dists,prod(dists))
        out=max(out,prod(dists))
print(out)

486540


7a

In [31]:
with open('in.txt') as f:
    lines=f.read().split('\n')
    cmd_data=[]
    i=0
    while i<len(lines):
        line=lines[i]
        assert line[0]=='$'
        cmd=line[2:]
        #print(cmd)
        if cmd.startswith('cd'):
            cmd_data.append((True,cmd[3:]))
            _dir=cmd[3:]
            #print(_dir)
            i+=1
        else:
            assert cmd=='ls'
            ret=[]
            i+=1
            while i<len(lines) and lines[i][0]!='$':
                ret.append(lines[i])
                i+=1
            cmd_data.append((False,ret))
'''class Node:
    #name,properties (file size or list of contents),par
    def __init__(name,props,par):
        self.name=name
        self.props=props
        self.par=par
root=None'''
tree={}
path=[]
for t,data in cmd_data:
    if t:
        if data=='..':
            path.pop()
        else:
            path.append(data)
    else:
        tree[tuple(path)]=data
sizes={}
def size(d):
    if d in sizes:
        return sizes[d]
    out=0
    for tok in tree[d]:
        if tok.startswith('dir '):
            out+=size(d+(tok[4:],))
        else:
            out+=int(tok.split(' ')[0])
    sizes[d]=out
    return out
for d in tree:
    size(d)
#print(sizes)
sum(s for d,s in sizes.items() if s<=100_000)

1642503

7b

In [30]:
cur_free=70000000-sizes[('/',)]
target=30000000
min(s for d,s in sizes.items() if cur_free+s>=target)

6999588

6a

In [7]:
with open('in.txt') as f:
    s=f.read()
    for i in range(len(s)-3):
        p=s[i:i+4]
        if len(set(list(p)))==len(p):
            print(i+4)
            break

1356


6b

In [8]:
with open('in.txt') as f:
    s=f.read()
    K=14
    for i in range(len(s)-K+1):
        p=s[i:i+K]
        if len(set(list(p)))==len(p):
            print(i+K)
            break

2564


5a

In [20]:
with open('in.txt') as f:
    out=0
    mvs=[]
    stacks=None
    for l in f.read().split('\n'):
        if l.startswith('move'):
            toks=l.split(' ')
            mvs.append(list(map(int,toks[1::2])))
        elif l.find('[')>=0:
            pcs=[l[i:i+4] for i in range(0,len(l),4)]
            if stacks==None:
                stacks=[[] for _ in range(len(pcs))]
            for i in range(len(stacks)):
                s=pcs[i].strip()
                if s!='':
                    ltr=s[1]
                    stacks[i].append(s)
    stacks=[st[::-1] for st in stacks]
    print(stacks)
    for cnt,i0,i1 in mvs:
        i0-=1
        i1-=1
        for _ in range(cnt):
            e=stacks[i0].pop()
            stacks[i1].append(e)
        #print(stacks)
    for st in stacks:
        print(st[-1][1],end='')

[['[D]', '[H]', '[N]', '[Q]', '[T]', '[W]', '[V]', '[B]'], ['[D]', '[W]', '[B]'], ['[T]', '[S]', '[Q]', '[W]', '[J]', '[C]'], ['[F]', '[J]', '[R]', '[N]', '[Z]', '[T]', '[P]'], ['[G]', '[P]', '[V]', '[J]', '[M]', '[S]', '[T]'], ['[B]', '[W]', '[F]', '[T]', '[N]'], ['[B]', '[L]', '[D]', '[Q]', '[F]', '[H]', '[V]', '[N]'], ['[H]', '[P]', '[F]', '[R]'], ['[Z]', '[S]', '[M]', '[B]', '[L]', '[N]', '[P]', '[H]']]
PSNRGBTFT

5b

In [21]:
with open('in.txt') as f:
    out=0
    mvs=[]
    stacks=None
    for l in f.read().split('\n'):
        if l.startswith('move'):
            toks=l.split(' ')
            mvs.append(list(map(int,toks[1::2])))
        elif l.find('[')>=0:
            pcs=[l[i:i+4] for i in range(0,len(l),4)]
            if stacks==None:
                stacks=[[] for _ in range(len(pcs))]
            for i in range(len(stacks)):
                s=pcs[i].strip()
                if s!='':
                    ltr=s[1]
                    stacks[i].append(s)
    stacks=[st[::-1] for st in stacks]
    print(stacks)
    for cnt,i0,i1 in mvs:
        i0-=1
        i1-=1
        gp=stacks[i0][-cnt:]
        stacks[i0]=stacks[i0][:-cnt]
        stacks[i1].extend(gp)
        #print(stacks)
    for st in stacks:
        print(st[-1][1],end='')

[['[D]', '[H]', '[N]', '[Q]', '[T]', '[W]', '[V]', '[B]'], ['[D]', '[W]', '[B]'], ['[T]', '[S]', '[Q]', '[W]', '[J]', '[C]'], ['[F]', '[J]', '[R]', '[N]', '[Z]', '[T]', '[P]'], ['[G]', '[P]', '[V]', '[J]', '[M]', '[S]', '[T]'], ['[B]', '[W]', '[F]', '[T]', '[N]'], ['[B]', '[L]', '[D]', '[Q]', '[F]', '[H]', '[V]', '[N]'], ['[H]', '[P]', '[F]', '[R]'], ['[Z]', '[S]', '[M]', '[B]', '[L]', '[N]', '[P]', '[H]']]
BNTZFPMMW

4a

In [3]:
with open('in.txt') as f:
    cnt=0
    for l in f.read().split('\n'):
        ps=l.split(',')
        rngs=list(map(lambda s:list(map(int,s.split('-'))),
                     ps))
        def subset(A,B):
            return B[0]<=A[0] and A[1]<=B[1]
        if subset(rngs[0],rngs[1]) or subset(rngs[1],rngs[0]):
            cnt+=1
    print(cnt)

536


4b

In [4]:
with open('in.txt') as f:
    cnt=0
    for l in f.read().split('\n'):
        ps=l.split(',')
        rngs=list(map(lambda s:list(map(int,s.split('-'))),
                     ps))
        def intx(A,B):
            return max(A[0],B[0])<=min(A[1],B[1])
        if intx(rngs[0],rngs[1]):
            cnt+=1
    print(cnt)

845


3a

In [9]:
Ps={
    **{chr(i+ord('a')):i+1 for i in range(26)},
    **{chr(i+ord('A')):i+27for i in range(26)}
}
with open('in.txt') as f:
    out=0
    for l in f.read().split('\n'):
        i=len(l)//2
        A,B=l[:i],l[i:]
        s=list(set(list(A))&set(list(B)))[0]
        #print(s)
        out+=Ps[s]
    print(out)

7795


3b

In [12]:
with open('in.txt') as f:
    ls=f.read().split('\n')
    out=0
    for li in range(0,len(ls),3):
        A,B,C=ls[li:li+3]
        s=list(set(list(A))&set(list(B))&set(list(C)))[0]
        #print(s)
        out+=Ps[s]
    print(out)

2703


2a

In [6]:
with open('in.txt') as f:
    scr=0
    for l in f.read().split('\n'):
        elf,me=l.split(' ')
        scr+=({'X':1,'Y':2,'Z':3}[me]
              +{
                  'X':{'A':3,'B':0,'C':6},
                  'Y':{'A':6,'B':3,'C':0},
                  'Z':{'A':0,'B':6,'C':3},
               }[me][elf])
    print(scr)

12740


2b

In [9]:
with open('in.txt') as f:
    scr=0
    for l in f.read().split('\n'):
        elf,ret=l.split(' ')
        me={
            'X':{'A':'Z','B':'X','C':'Y'},
            'Y':{'A':'X','B':'Y','C':'Z'},
            'Z':{'A':'Y','B':'Z','C':'X'},
        }[ret][elf]
        scr+=({'X':1,'Y':2,'Z':3}[me]
              +{
                  'X':{'A':3,'B':0,'C':6},
                  'Y':{'A':6,'B':3,'C':0},
                  'Z':{'A':0,'B':6,'C':3},
               }[me][elf])
    print(scr)

11980


1a

In [13]:
with open('in.txt') as f:
    groups=[list(map(int,g.split('\n')))
            for g in f.read().split('\n\n')]
    #print(groups)
    print(max(map(sum,groups)))

75501


1b

In [14]:
print(sum(sorted(map(sum,groups),reverse=True)[:3]))

215594
